In [6]:
import sys
!{sys.executable} -m pip install shapely


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.7 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: /opt/homebrew/Cellar/jupyterlab/4.3.5/libexec/bin/python -m pip install --upgrade pip


In [5]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension --sys-prefix


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
Using cached ipywidgets-8.1.7-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl (216 kB)
Using cached widgetsnbextension-4.0.14-py3-none-any.whl (2.2 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.2/808.2 kB 11.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.6 MB/s  0:00:00
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, pl

In [ ]:
Refactor this python code so in arram boyle mode the code creates a polygonarcgroup object containing the entire closed area outline formed by arcs. I later want to use that to clip a background I want to fill into the arc. For now just refactor without background while keeping all functionality the same. The issue is the arcs from one circle will be in different groups and multiple times in multiple groups. So best would be to go over all arcs and recognize arcs where the ends intersect (with a error margin) and a circle of such arcs is an enclosed area. Only arc groups of length 8 are relevant. You can use the fact constructing it that each arc group consists of the four arcs kept from each circle. And then one arc each from the neighbouring two circles above and below. Implement it elegantly, readable and object oriented, possibly extending existing classes. Now the creation of the arc groups I will do manually, please prepare the code for this. I will do it via circle properties. I need to be able to access a circles 6 neighbour circles and the arcs to match the arcs to a group properly. 

In [7]:
%matplotlib inline
from IPython.display import display, SVG
from matplotlib.path import Path as MplPath
import ipywidgets as widgets
import math, random
from pathlib import Path
import numpy as np
from scipy.spatial import Voronoi
import colorsys

# ----------------- Utilities -----------------
def hsv_to_hex(h, s, v):
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    return f'#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}'

def voronoi_finite_polygons_2d(vor, radius=None):
    """Convert infinite Voronoi regions to finite polygons."""
    if vor.points.shape[1] != 2:
        raise ValueError("Requires 2D input")

    new_regions = []
    new_vertices = vor.vertices.tolist()

    center = vor.points.mean(axis=0)
    if radius is None:
        radius = vor.points.ptp().max() * 2

    all_ridges = {}
    for (p1, p2), (v1, v2) in zip(vor.ridge_points, vor.ridge_vertices):
        all_ridges.setdefault(p1, []).append((p2, v1, v2))
        all_ridges.setdefault(p2, []).append((p1, v1, v2))

    for p1, region_idx in enumerate(vor.point_region):
        vertices = vor.regions[region_idx]
        if vertices is None or len(vertices) == 0:
            new_regions.append([])
            continue
        if all(v >= 0 for v in vertices):
            new_regions.append(vertices)
            continue

        ridges = all_ridges.get(p1, [])
        new_region = [v for v in vertices if v >= 0]

        for p2, v1, v2 in ridges:
            if v2 < 0 or v1 < 0:
                v_finite = v1 if v1 >= 0 else v2
                tangent = vor.points[p2] - vor.points[p1]
                nrm = np.linalg.norm(tangent)
                if nrm == 0:
                    continue
                tangent /= nrm
                normal = np.array([-tangent[1], tangent[0]])
                midpoint = vor.points[[p1, p2]].mean(axis=0)
                direction = np.sign(np.dot(midpoint - center, normal)) * normal
                far_point = vor.vertices[v_finite] + direction * radius
                new_vertices.append(far_point.tolist())
                new_region.append(len(new_vertices) - 1)

        if len(new_region) == 0:
            new_regions.append([])
            continue

        vs = np.asarray([new_vertices[v] for v in new_region])
        c = vs.mean(axis=0)
        angles = np.arctan2(vs[:, 1] - c[1], vs[:, 0] - c[0])
        new_region = [v for _, v in sorted(zip(angles, new_region))]
        new_regions.append(new_region)

    return new_regions, np.asarray(new_vertices)


def phyllotaxis(n_points, width, height, turns=10, spacing_power=1.6, margin=0.05, single_spiral=False):
    """Generate a golden-ratio or single outward spiral of points."""
    max_r = 0.5 * min(width, height) * (1 - margin)
    pts = []

    if single_spiral:
        total_angle = turns * 2 * math.pi
        for i in range(n_points):
            theta = i / (n_points - 1) * total_angle if n_points > 1 else 0
            ratio = (i / (n_points - 1)) ** spacing_power if n_points > 1 else 0
            r = max_r * ratio
            x = r * math.cos(theta)
            y = r * math.sin(theta)
            pts.append((x, y))
    else:
        golden_angle = math.pi * (3 - math.sqrt(5))
        for i in range(n_points):
            theta = i * golden_angle
            ratio = (i / (n_points - 1)) ** spacing_power if n_points > 1 else 0
            r = max_r * math.sqrt(ratio)
            x = r * math.cos(theta)
            y = r * math.sin(theta)
            pts.append((x, y))

    pts = np.array(pts, dtype=float)
    pts[:, 0] += width / 2
    pts[:, 1] += height / 2
    return pts


def polygon_to_svg(points, fill, stroke='#000', stroke_width=0.5, fill_opacity=1.0):
    coords = ' '.join(f'{x:.3f},{y:.3f}' for x, y in points)
    return f'<polygon points="{coords}" fill="{fill}" fill-opacity="{fill_opacity}" stroke="{stroke}" stroke-width="{stroke_width}" />\n'


def lines_in_polygon(polygon, line_spacing=5, angle=0, color="#0000ff", stroke_width=0.5):
    """Fill polygon with parallel lines at a given angle. polygon: Nx2 numpy array or list."""
    polygon = np.array(polygon)
    if polygon.shape[0] < 3:
        return ""
    # bounding box / diag for sampling length
    min_x, min_y = polygon[:, 0].min(), polygon[:, 1].min()
    max_x, max_y = polygon[:, 0].max(), polygon[:, 1].max()
    bbox_diag = math.hypot(max_x - min_x, max_y - min_y)
    if bbox_diag == 0:
        return ""

    centroid = polygon.mean(axis=0)
    theta = math.radians(angle)
    dx, dy = math.cos(theta), math.sin(theta)
    # perpendicular direction for shifting parallel lines
    px, py = -dy, dx

    # number of lines needed to cover bounding box
    # increase multiplier to be safe for thin polygons
    num_lines = int(bbox_diag / max(1e-6, line_spacing)) + 3

    path = MplPath(polygon)
    lines_svg = []

    # ensure we sample at least a few points per candidate line
    sample_length = int(bbox_diag * 2)  # length across which we sample points on each line
    sample_length = max(sample_length, 40)  # ensure decent resolution

    for i in range(-num_lines, num_lines + 1):
        shift_x = px * i * line_spacing
        shift_y = py * i * line_spacing
        line_start = np.array([centroid[0] - dx * sample_length + shift_x,
                               centroid[1] - dy * sample_length + shift_y])
        line_end = np.array([centroid[0] + dx * sample_length + shift_x,
                             centroid[1] + dy * sample_length + shift_y])

        xs = np.linspace(line_start[0], line_end[0], sample_length)
        ys = np.linspace(line_start[1], line_end[1], sample_length)
        inside = [path.contains_point((x, y)) for x, y in zip(xs, ys)]

        start_idx = None
        for j, inside_flag in enumerate(inside):
            if inside_flag and start_idx is None:
                start_idx = j
            elif not inside_flag and start_idx is not None:
                x1, y1 = xs[start_idx], ys[start_idx]
                x2, y2 = xs[j - 1], ys[j - 1]
                lines_svg.append(f'<line x1="{x1:.3f}" y1="{y1:.3f}" x2="{x2:.3f}" y2="{y2:.3f}" '
                                 f'stroke="{color}" stroke-width="{stroke_width}" />\n')
                start_idx = None
        # if a segment continues to the end
        if start_idx is not None:
            x1, y1 = xs[start_idx], ys[start_idx]
            x2, y2 = xs[-1], ys[-1]
            lines_svg.append(f'<line x1="{x1:.3f}" y1="{y1:.3f}" x2="{x2:.3f}" y2="{y2:.3f}" '
                             f'stroke="{color}" stroke-width="{stroke_width}" />\n')

    return ''.join(lines_svg)

# ----------------- SVG Generator -----------------
def make_svg(n_points=300, width=800, height=800, spacing_power=1.6, turns=10,
             outlines_only=False, add_lines=False, line_angle_shift=15, line_spacing=5,
             single_spiral=False, extend_voronoi=False, extra_voronoi_fraction=0.3, extra_voronoi_turns=3,
             only_outer_red=False, display_path=False):

    if extend_voronoi:
        n_total = max(n_points + 1, int(n_points * (1.0 + extra_voronoi_fraction)))
        pts_all = phyllotaxis(n_total, width, height, turns + extra_voronoi_turns, spacing_power, single_spiral=single_spiral)
        points = pts_all
        real_n = n_points
    else:
        points = phyllotaxis(n_points, width, height, turns, spacing_power, single_spiral=single_spiral)
        real_n = n_points

    xmin, ymin, xmax, ymax = 0.0, 0.0, float(width), float(height)

    def clip_polygon_to_bbox(poly):
        def intersection(a, b, axis, val):
            ax, ay = a
            bx, by = b
            if axis == 'x':
                if bx == ax: return (val, ay)
                t = (val - ax) / (bx - ax)
                return (val, ay + t * (by - ay))
            else:
                if by == ay: return (ax, val)
                t = (val - ay) / (by - ay)
                return (ax + t * (bx - ax), val)

        def clip_with_edge(points, axis, val, keep_greater):
            new_pts = []
            n = len(points)
            for i in range(n):
                a = points[i]
                b = points[(i + 1) % n]
                if axis == 'x':
                    a_in = a[0] >= val if keep_greater else a[0] <= val
                    b_in = b[0] >= val if keep_greater else b[0] <= val
                else:
                    a_in = a[1] >= val if keep_greater else a[1] <= val
                    b_in = b[1] >= val if keep_greater else b[1] <= val
                if a_in and b_in:
                    new_pts.append(b)
                elif a_in and not b_in:
                    new_pts.append(intersection(a, b, axis, val))
                elif not a_in and b_in:
                    new_pts.append(intersection(a, b, axis, val))
                    new_pts.append(b)
            return new_pts

        pts = [tuple(p) for p in poly]
        pts = clip_with_edge(pts, 'x', xmin, True)
        pts = clip_with_edge(pts, 'x', xmax, False)
        pts = clip_with_edge(pts, 'y', ymin, True)
        pts = clip_with_edge(pts, 'y', ymax, False)
        return np.array(pts)

    svg_elements = [
        f'<svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 {width} {height}" width="{width}" height="{height}">\n',
        f'<rect x="0" y="0" width="{width}" height="{height}" fill="#ffffff"/>\n'
    ]

    random.seed(42)
    current_angle = 0.0

    vor = Voronoi(points)
    regions, vertices = voronoi_finite_polygons_2d(vor, radius=max(width, height) * 2)

    # ----------------- Compute clipped polygons -----------------
    clipped_polygons_data = []
    for idx in range(min(real_n, len(regions))):
        region = regions[idx]
        if not region or len(region) == 0:
            continue
        polygon = vertices[region]
        clipped = clip_polygon_to_bbox(polygon)
        if len(clipped) < 3:
            continue
        clipped_polygons_data.append((idx, clipped))

    # ----------------- Count edges to find outer edges -----------------
    from collections import defaultdict
    edge_counts = defaultdict(int)
    for _, clipped in clipped_polygons_data:
        n = len(clipped)
        for i in range(n):
            a, b = tuple(clipped[i]), tuple(clipped[(i + 1) % n])
            edge_counts[tuple(sorted([a, b]))] += 1

    # ----------------- Draw polygons with outer edges colored -----------------
    for idx, clipped in clipped_polygons_data:
        # Fill color
        if outlines_only:
            fill = "none"
        else:
            h = (idx / max(1, n_points)) % 1.0
            fill = hsv_to_hex(h, 0.5, 0.9)

        # Draw edges individually
        n = len(clipped)
        for i in range(n):
            a, b = tuple(clipped[i]), tuple(clipped[(i + 1) % n])
            edge_key = tuple(sorted([a, b]))
            stroke_color = "#ff0000" if (only_outer_red and edge_counts[edge_key] == 1) else "#000000"
            svg_elements.append(f'<line x1="{a[0]:.3f}" y1="{a[1]:.3f}" '
                                f'x2="{b[0]:.3f}" y2="{b[1]:.3f}" '
                                f'stroke="{stroke_color}" stroke-width="0.7" />\n')

    # ----------------- Optional: hatch lines inside polygons -----------------
    if add_lines:
        current_angle = 0.0
        for _, clipped in clipped_polygons_data:
            current_angle = (current_angle + line_angle_shift) % 360
            svg_elements.append(lines_in_polygon(np.array(clipped),
                                                 line_spacing=line_spacing,
                                                 angle=current_angle,
                                                 color="#000000",
                                                 stroke_width=0.35))

    # ----------------- Spiral path display -----------------
    if display_path:
        if single_spiral:
            path_points = points
        else:
            center = np.array([width / 2, height / 2])
            angles = np.arctan2(points[:, 1] - center[1], points[:, 0] - center[0])
            sort_idx = np.argsort(angles)
            path_points = points[sort_idx]

        path_coords = ' '.join(f'{x:.3f},{y:.3f}' for x, y in path_points)
        svg_elements.append(f'<polyline points="{path_coords}" fill="none" stroke="#ff0000" stroke-width="0.8" opacity="0.6"/>\n')

        for (x, y) in points:
            svg_elements.append(f'<circle cx="{x:.3f}" cy="{y:.3f}" r="2" fill="#0000ff" opacity="0.8"/>\n')

    svg_elements.append('</svg>')
    return ''.join(svg_elements)

 
    
# ----------------- Interactive UI -----------------
def show_ui():
    mode_label = widgets.Label("Voronoi / Phyllotaxis")
    n_points_slider = widgets.IntSlider(value=400, min=50, max=1500, step=50, description='N Points')
    spacing_slider = widgets.FloatSlider(value=1.6, min=0.2, max=3.0, step=0.05, description='Spacing Power')
    turns_slider = widgets.FloatSlider(value=10, min=1, max=40, step=1, description='Turns')
    outlines_toggle = widgets.Checkbox(value=False, description='Outlines Only')
    outer_red_toggle = widgets.Checkbox(value=False, description='Only Outer Line Red')
    add_lines_toggle = widgets.Checkbox(value=False, description='Add Lines')
    single_spiral_toggle = widgets.Checkbox(value=False, description='Single Spiral')
    display_path_toggle = widgets.Checkbox(value=False, description='Display Spiral Path')
    line_angle_shift_slider = widgets.IntSlider(value=15, min=1, max=60, step=1, description='Angle Shift')
    line_spacing_slider = widgets.IntSlider(value=5, min=1, max=40, step=1, description='Line Spacing')

    extend_voronoi_toggle = widgets.Checkbox(value=False, description='Extend Voronoi')
    extra_voronoi_fraction_slider = widgets.FloatSlider(value=0.3, min=0.0, max=1.0, step=0.05, description='Extra Fraction')
    extra_voronoi_turns_slider = widgets.IntSlider(value=3, min=0, max=10, step=1, description='Extra Turns')

    save_button = widgets.Button(description='Save SVG')
    output = widgets.Output(layout={'border': '1px solid black'})

    ui = widgets.VBox([
        mode_label,
        n_points_slider, spacing_slider, turns_slider,
        outlines_toggle, outer_red_toggle,
        add_lines_toggle, single_spiral_toggle, display_path_toggle,
        line_angle_shift_slider, line_spacing_slider,
        extend_voronoi_toggle, extra_voronoi_fraction_slider, extra_voronoi_turns_slider,
        save_button, output
    ])

    def update(*args):
        with output:
            output.clear_output()
            svg_str = make_svg(
                n_points=n_points_slider.value,
                spacing_power=spacing_slider.value,
                turns=turns_slider.value,
                outlines_only=outlines_toggle.value,
                add_lines=add_lines_toggle.value,
                single_spiral=single_spiral_toggle.value,
                line_angle_shift=line_angle_shift_slider.value,
                line_spacing=line_spacing_slider.value,
                extend_voronoi=extend_voronoi_toggle.value,
                extra_voronoi_fraction=extra_voronoi_fraction_slider.value,
                extra_voronoi_turns=extra_voronoi_turns_slider.value,
                only_outer_red=outer_red_toggle.value,
                display_path=display_path_toggle.value
            )
            display(SVG(svg_str))

    controls = [n_points_slider, spacing_slider, turns_slider, outlines_toggle, outer_red_toggle,
                add_lines_toggle, single_spiral_toggle, display_path_toggle, line_angle_shift_slider, line_spacing_slider,
                extend_voronoi_toggle, extra_voronoi_fraction_slider, extra_voronoi_turns_slider]

    for w in controls:
        w.observe(update, 'value')

    def save_svg(b):
        svg_str = make_svg(
            n_points=n_points_slider.value,
            spacing_power=spacing_slider.value,
            turns=turns_slider.value,
            outlines_only=outlines_toggle.value,
            add_lines=add_lines_toggle.value,
            single_spiral=single_spiral_toggle.value,
            line_angle_shift=line_angle_shift_slider.value,
            line_spacing=line_spacing_slider.value,
            extend_voronoi=extend_voronoi_toggle.value,
            extra_voronoi_fraction=extra_voronoi_fraction_slider.value,
            extra_voronoi_turns=extra_voronoi_turns_slider.value,
            only_outer_red=outer_red_toggle.value,
            display_path=display_path_toggle.value
        )
        Path('output.svg').write_text(svg_str)
        with output:
            print("SVG saved as output.svg")

    save_button.on_click(save_svg)
    display(ui)
    update()


show_ui()
